In [ ]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import osmnx as ox
from pathlib import Path

In [ ]:
def readStatelog(folder, file):
    return pd.read_csv(os.path.join(folder,file), skiprows = 1)

def readGeneralLog(folder, file):
    conflicts_ref = pd.read_csv(os.path.join(folder,file), skiprows = 3)
    conflicts_ref = conflicts_ref.drop([0])
    return conflicts_ref

def readFolder(folder):
    files_dict = {}
    for file in os.listdir(folder):
        if file.split('_')[0] == 'STATELOG':
            statelog = readStatelog(folder, file)
            files_dict['USEPESTATELOG'] = statelog
        else:
            files_dict[file.split('_')[0]] = readGeneralLog(folder, file)
    
    return files_dict

def distnace(lat1,lon1,lat2,lon2):
    lat1 = lat1 * math.pi/180
    lat2 = lat2 * math.pi/180
    lon1 = lon1 * math.pi/180
    lon2 = lon2 * math.pi/180
    
    return math.acos(math.sin(lat1)*math.sin(lat2)+math.cos(lat1)*math.cos(lat2)*math.cos(lon2-lon1))*6371

def readLogs(folder):
    files_dict = readFolder(folder)
        
    conflog = files_dict['USEPECONFLOG']
    cpalog = files_dict['USEPECPALOG']
    flightlog = files_dict['USEPEFLIGHTLOG']
    loslog = files_dict['USEPELOSLOG']
    statelog = files_dict['USEPESTATELOG']
    
    return statelog, conflog, loslog, cpalog, flightlog

# Analysing flight log

In [ ]:
def analyseFlightLog(flightlog):
    flightlog['# Simulation Time [s]'] = flightlog['# Simulation Time [s]'].astype(float)

    flightlog = flightlog[['SURVEILLANCE_' not in x for x in flightlog[' UAS']]]

    flightlog_first = flightlog.groupby(' UAS').first()
    started_flights = flightlog_first[flightlog_first[' Takeoff/Landing'] == 'takeoff'].index

    flightlog_last = flightlog.groupby(' UAS').last()
    completed_flights = flightlog_last[flightlog_last[' Takeoff/Landing'] == 'landing'].index

    print("Started flights: {}".format(len(started_flights)))
    print("Completed flights: {}".format(len(completed_flights)))
    
    flightlog_landed = flightlog[flightlog[' UAS'].isin(completed_flights)]
    total_distance = sum(flightlog_landed.groupby(' UAS').sum()[' Distance [m]'])
    
    avg_distance_flown = total_distance / len(completed_flights)

    print( "Average distance flown: {}".format(avg_distance_flown ))
    
    df = flightlog[flightlog[' UAS'].isin(completed_flights)].groupby(' Takeoff/Landing').sum()
    total_time = df.loc['landing']['# Simulation Time [s]'] - df.loc['takeoff']['# Simulation Time [s]']
    
    avg_flight_time = total_time / len(completed_flights)

    print( "Average flight time: {}".format(avg_flight_time ))
    
    return len(started_flights), len(completed_flights), avg_distance_flown, avg_flight_time
    

# Analysing conflicts

In [ ]:
def analyseConflicts(conflog, threshold=20):
    conflog['# Simulation Time [s]'] = conflog['# Simulation Time [s]'].astype(float)
    
    conflog_filtered = filterConflict(conflog, threshold)
    
    print("Number of conflicts: {}".format(len(conflog_filtered)))
    print("Average conflict time: {}".format(conflog_filtered['time_in_conflict'].mean()))
    
    return len(conflog_filtered), conflog_filtered['time_in_conflict'].mean(), conflog_filtered

def filterConflict(conflog, threshold):
    conflicts_dict = {}
    conflicts_dict['UAS1'] = []
    conflicts_dict['UAS2'] = []
    conflicts_dict['start'] = []
    conflicts_dict['end'] = []
    conflicts_dict['time_in_conflict'] = []

    for UAS1 in conflog[' UAS1'].unique():
        df = conflog[conflog[' UAS1'] == UAS1]
        for UAS2 in df[' UAS2'].unique():
            df2 = df[df[' UAS2'] == UAS2] 
            start = True
            time_in_conflict = 0
            for idx, row in df2.iterrows():

                if row[' Start/End'] == 'start':
                    if start:
                        start_time = row['# Simulation Time [s]']
                        prev_start_time = row['# Simulation Time [s]']
                        start = False
                    else:
                        if row['# Simulation Time [s]'] > end_time + threshold:
                            conflicts_dict['UAS1'] += [UAS1]
                            conflicts_dict['UAS2'] += [UAS2]
                            conflicts_dict['start'] += [start_time]
                            conflicts_dict['end'] += [end_time]
                            conflicts_dict['time_in_conflict'] += [time_in_conflict]


                            time_in_conflict = 0
                            start_time = row['# Simulation Time [s]']
                            prev_start_time = row['# Simulation Time [s]']

                        else:
                            prev_start_time = row['# Simulation Time [s]'] 
                    completed = False
                else:
                    end_time = row['# Simulation Time [s]']
                    time_in_conflict += row['# Simulation Time [s]'] - prev_start_time
                    completed = True

            if completed:
                conflicts_dict['UAS1'] += [UAS1]
                conflicts_dict['UAS2'] += [UAS2]
                conflicts_dict['start'] += [start_time]
                conflicts_dict['end'] += [end_time]
                conflicts_dict['time_in_conflict'] += [time_in_conflict]



    conflog_filtered = pd.DataFrame.from_dict(conflicts_dict)
    return conflog_filtered

def filterDeliveryPoint(conflog_filtered, statelog, threshold = 0.25):
    df = pd.DataFrame()
    for idx, row in conflog_filtered.iterrows():
        if ('DELIVERY' in row['UAS1']) or ('DELIVERY' in row['UAS2']):
            #time = math.ceil((row['end'] + row['start'])/2)
            time = row['start']        

            uas1 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS1'] ) ].iloc[0]
            uas2 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS2'] ) ].iloc[0]

            lat_ref = 52.3594
            lon_ref = 9.7499

            lat1 = uas1[' lat']
            lon1 = uas1[' lon']
            lat2 = uas2[' lat']
            lon2 = uas2[' lon']

            d1 = distnace(lat_ref,lon_ref,lat1,lon1)
            d2 = distnace(lat_ref,lon_ref,lat2,lon2)

            if (d1 < threshold) or (d2 < threshold):
                delivery_region = True
            else:
                delivery_region = False
                df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
        else:
            delivery_region = False
            df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
            
    return df
        

# Analysing LoS

In [ ]:
def analyseLoS(loslog, threshold=20):
    loslog['# Simulation Time [s]'] = loslog['# Simulation Time [s]'].astype(float)
    
    loslog_filtered = filterConflict(loslog, threshold)
    
    print("Number of LoS: {}".format(len(loslog_filtered)))
    print("Average LoS time: {}".format(loslog_filtered['time_in_conflict'].mean()))

    return len(loslog_filtered), loslog_filtered['time_in_conflict'].mean(), loslog_filtered

# Analysing CPA

In [ ]:
def analyseCpa(cpalog, threshold=20):
    cpalog['# Simulation Time [s]'] = cpalog['# Simulation Time [s]'].astype(float)
    cpalog_filtered = filterCpa(cpalog, threshold)
    
    NMAC = cpalog_filtered[cpalog_filtered['distance'] < 3.75].copy().reset_index(drop=True)
    
    print("Number of NMACs: {}".format(len(NMAC)))
    
    return len(NMAC), NMAC
    
def filterCpa(cpalog, threshold):
    conflicts_dict = {}
    conflicts_dict['UAS1'] = []
    conflicts_dict['UAS2'] = []
    conflicts_dict['distance'] = []
    conflicts_dict['time'] = []

    for UAS1 in cpalog[' UAS1'].unique():
        df = cpalog[cpalog[' UAS1'] == UAS1]
        for UAS2 in df[' UAS2'].unique():
            df2 = df[df[' UAS2'] == UAS2]
            
            start = True
            distance = 99
            for idx, row in df2.iterrows():
                if start:
                    start_time = row['# Simulation Time [s]']
                    distance = row[' Distance [m]']
                    time = row['# Simulation Time [s]']
                    start = False
                else:
                    if row['# Simulation Time [s]'] > start_time + threshold:
                        conflicts_dict['UAS1'] += [UAS1]
                        conflicts_dict['UAS2'] += [UAS2]
                        conflicts_dict['distance'] += [distance]
                        conflicts_dict['time'] += [time]

                        start_time = row['# Simulation Time [s]']
                        distance = row[' Distance [m]']
                        time = row['# Simulation Time [s]']

                    else:
                        if row[' Distance [m]'] < distance:
                            distance = row[' Distance [m]']
                            time = row['# Simulation Time [s]']
                        start_time = row['# Simulation Time [s]']
                            
                        prev_start_time = row['# Simulation Time [s]'] 
            conflicts_dict['UAS1'] += [UAS1]
            conflicts_dict['UAS2'] += [UAS2]
            conflicts_dict['distance'] += [distance]
            conflicts_dict['time'] += [time]




    cpalog_filtered = pd.DataFrame.from_dict(conflicts_dict)
    return cpalog_filtered

def filterCPADeliveryPoint(conflog_filtered, statelog, threshold = 0.25):
    df = pd.DataFrame()
    for idx, row in conflog_filtered.iterrows():
        if ('DELIVERY' in row['UAS1']) or ('DELIVERY' in row['UAS2']):
            time = row['time']        
            try:
                uas1 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS1'] ) ].iloc[0]
                lat1 = uas1[' lat']
                lon1 = uas1[' lon']
            except:
                print(time)
                print(statelog[ (statelog[' id'] == row['UAS1'] ) ])
                lat1 = 0
                lon1 = 0
                
            try:
                uas2 = statelog[(statelog['# simt'] == time) & (statelog[' id'] == row['UAS2'] ) ].iloc[0]
                lat2 = uas2[' lat']
                lon2 = uas2[' lon']
            except:
                print(time)
                print(statelog[ (statelog[' id'] == row['UAS2'] ) ])
                lat2 = 0
                lon2 = 0

            lat_ref = 52.3594
            lon_ref = 9.7499

            d1 = distnace(lat_ref,lon_ref,lat1,lon1)
            d2 = distnace(lat_ref,lon_ref,lat2,lon2)

            if (d1 < threshold) or (d2 < threshold):
                delivery_region = True
            else:
                delivery_region = False
                df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
        else:
            delivery_region = False
            df = pd.concat([df, pd.DataFrame(row).T]).reset_index(drop=True)
            
    return df

# Batch analysis
### All the experimentes are analised at the same time. First we have to define all the folders where the logs files are stored. The final output is a a DataFrame with all the metrics

In [ ]:
##### Definded by user ######

# We create the paths to all the folders where the logs are stored
use_case = 2

if use_case == 1:
    # EXERCISE 1 #
    densities = ['low', 'medium']
    #exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9', '0_no_wind', '1_no_wind', '2_no_wind']
    exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9']
    exp_high = ['0']
    #exp = ['0_no_wind', '1_no_wind', '2_no_wind']
    mode = ['ref', 'D2C2']
    folder_path = r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP5 Simulation and Validation\00 Last mile delivery use case\03. BlueSky outputs\{}_density\exp_{}\{}"
    code = [0,1,2]
    threshold_distance = 0.1 # distance to filter the conflicts near the delivery point
    # Region under study
    lat_min = 52.33
    lat_max = 52.4
    lon_min = 9.71
    lon_max = 9.775
    # Duration of simulation
    duration = 3600
elif use_case == 2:
    # EXERCISE 2 # 
    densities = ['low', 'medium']
    exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9']
    exp_high = ['0']
    mode = ['ref', 'D2C2']
    folder_path = r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP5 Simulation and Validation\01 Emergency drone use case\03. BlueSky outputs\{}_density\exp_{}\{}"
    code = [0,1,2]
    threshold_distance = 0 # distance to filter the conflicts near the delivery point
    # Region under study
    lat_min = 52.31
    lat_max = 52.385
    lon_min = 9.70
    lon_max = 9.78
    # Duration of simulation
    duration = 3600
elif use_case == 3:
    # EXERCISE 3 # 
    densities = ['low', 'medium']
    exp = ['1','2', '3', '4', '5', '6', '7', '8', '9', '10']
    exp_high = ['1', '4', '5', '6', '10']
    mode = ['ref', 'd2c2']
    folder_path = r"/home/ror/ws/scenarios/exercise_3/output/{}_density/dataset_{}/{}/reduced_capacity"
    code = [0,1,2]
    threshold_distance = 0
    # Region under study
    lat_min = 52.412274
    lat_max = 52.466148
    lon_min = 9.638119
    lon_max = 9.745655
    # Duration of simulation
    duration = 1800
else: 
    # Improvements #
    densities = ['improvements']
    exp = ['0','1','2', '3', '4', '5', '6', '7', '8', '9']
    mode = []
    folder_path = r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2021 USEPE\iii) Project\WP5 Simulation and Validation\00 Last mile delivery use case\03. BlueSky outputs\{}_density\exp_{}\{}"
    code = [0,1,2]

exercises = [(x,y,z) for x in densities for y in exp for z in mode]
exercises_high = [('high',y,z) for y in exp_high for z in mode]
exercises += exercises_high

folder_list = []
for exe in exercises:
    a = exe[0]
    b = exe[1]
    c = exe[2]

    folder = folder_path.format(exe[code[0]],exe[code[1]],exe[code[2]])
    folder_list += [folder]

In [ ]:
l1 = ox.distance.great_circle_vec(lat_min,lon_min,lat_min,lon_max)
l2 = ox.distance.great_circle_vec(lat_min,lon_min,lat_max,lon_min)

print(l1)
print(l2)
print("Region area {} km2".format(l1 / 1000 * l2 / 1000))

In [ ]:
folder_list

In [ ]:
results = {}
results['Density'] = []
results['Dataset'] = []
results['Scenario'] = []
results['Started Flights'] = []
results['Completed Flights'] = []
results['Avg. Distance Flown'] = []
results['Avg. flying time'] = []
results['Conflicts'] = []
results['Avg. conflict time'] = []
results['LoS'] = []
results['NMACs'] = []
if use_case == 1:
    results['Conflicts (without delivery point)'] = []
    results['Avg. conflict time (without delivery point)'] = []
    results['LoS (without delivery point)'] = []
    results['NMACs (without delivery point)'] = []

results['Simultaneous fligths'] = []

for folder, exe in zip(folder_list, exercises):
    
    a = exe[0]
    b = exe[1]
    c = exe[2]
    
    print(' ')
    print(folder)
    try:
        files_dict = readFolder(folder)
    except FileNotFoundError:
        continue
    
    print(' ')
    print([a,b,c])
        
    if not files_dict:
        continue
        
    conflog = files_dict['USEPECONFLOG']
    cpalog = files_dict['USEPECPALOG']
    flightlog = files_dict['USEPEFLIGHTLOG']
    loslog = files_dict['USEPELOSLOG']
    statelog = files_dict['USEPESTATELOG']
    
    started_flights, completed_flights, avg_distance_flown, avg_flight_time = analyseFlightLog(flightlog)
    
    conflicts, avg_time_in_conflict, conflog_filtered = analyseConflicts(conflog, threshold=20)
    
    conflog_filtered_delivery = filterDeliveryPoint(conflog_filtered, statelog, threshold = threshold_distance)
    
    los, _ , loslog_filtered = analyseLoS(loslog, threshold=20)
    
    loslog_filtered_delivery = filterDeliveryPoint(loslog_filtered, statelog, threshold = threshold_distance)
    
    NMAC, NMAC_df = analyseCpa(cpalog, threshold=20)
    
    NMAC_df_delivery = filterCPADeliveryPoint(NMAC_df, statelog, threshold = threshold_distance)
    
    results['Density'] += [a]
    results['Dataset'] += [b]
    results['Scenario'] += [c]
    results['Started Flights'] += [started_flights]
    results['Completed Flights'] += [completed_flights]
    results['Avg. Distance Flown'] += [avg_distance_flown]
    results['Avg. flying time'] += [avg_flight_time]
    results['Conflicts'] += [conflicts]
    results['Avg. conflict time'] += [avg_time_in_conflict]
    
    
    results['LoS'] += [los]
    
    
    results['NMACs'] += [NMAC]
    
    if use_case == 1:
        results['Conflicts (without delivery point)'] += [len(conflog_filtered_delivery)]
        results['Avg. conflict time (without delivery point)'] += [conflog_filtered_delivery['time_in_conflict'].mean()]
        results['LoS (without delivery point)'] += [len(loslog_filtered_delivery)]
        results['NMACs (without delivery point)'] += [len(NMAC_df_delivery)]
    
    results['Simultaneous fligths'] += [statelog.groupby(['# simt']).count().mean()[' id']]
    
df_result = pd.DataFrame.from_dict(results)

In [ ]:
df_result.sort_values(by=['Density', 'Scenario', 'Dataset'])

# Additional analysis

In [ ]:
def addLatLonAlt(dataset, statelog, metric):
    dataset = dataset[dataset['# Simulation Time [s]'].astype(float) <= duration]
    
    if metric == 'cpa':
        metric = ' Distance [m]'
        group = dataset.groupby([' UAS1', ' UAS2'], as_index=False)[metric].min().iterrows()
    elif metric == 'los':
        metric = ' Start/End'
        group = dataset.groupby([' UAS1', ' UAS2'], as_index=False)[metric].max().iterrows()
    elif metric == 'conf':
        metric = ' Start/End'
        group = dataset.groupby([' UAS1', ' UAS2'], as_index=False)[metric].max().iterrows()
    
    new_dataset = {}
    new_dataset['# Simulation Time [s]'] = []
    new_dataset[' UAS1'] = []
    new_dataset[' UAS2'] = []
    new_dataset[metric] = []
    new_dataset[' lat'] = []
    new_dataset[' lon'] = []
    new_dataset[' alt'] = []
    
    for idx, row in group:
        drone1 = row[' UAS1']
        drone2 = row[' UAS2']
        aux = row[metric]
        time = float(dataset[(dataset[' UAS1'] == drone1) & (dataset[metric] == aux)]['# Simulation Time [s]'].iloc[0])
        # Save original dataset data
        new_dataset['# Simulation Time [s]'] += [time]
        new_dataset[' UAS1'] += [drone1]
        new_dataset[' UAS2'] += [drone2]
        new_dataset[metric] += [aux]
        # Add lat lon alt
        try:
            new_dataset[' lat'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' lat'].iloc[0] ]
            new_dataset[' lon'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' lon'].iloc[0] ]
            new_dataset[' alt'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone1)][' alt'].iloc[0] ]
        except IndexError:
            new_dataset[' lat'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone2)][' lat'].iloc[0] ]
            new_dataset[' lon'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone2)][' lon'].iloc[0] ]
            new_dataset[' alt'] += [ statelog[(statelog['# simt'] == time) & (statelog[' id'] == drone2)][' alt'].iloc[0] ]
        
    df_new_dataset = pd.DataFrame.from_dict(new_dataset)
    return df_new_dataset

def returnAggregatedDatasets(density, mode, folder_list, exercises):
    statelog_frames = []
    flightlog_frames = []
    cpalog_frames = []
    conflog_frames = []
    loslog_frames = []
    
    for folder, exe in zip(folder_list, exercises):
    
        a = exe[0]
        b = exe[1]
        c = exe[2]
        
        #print(a, density, ' ', c, mode)
        
        if a == density and c == mode:
            
            #print(' ')
            #print(folder)
            try:
                files_dict = readFolder(folder)
            except FileNotFoundError:
                continue

            #print(' ')
            #print([a,b,c])

            if not files_dict:
                continue
            
            # Create data agrregating all the cases
            print(folder)
            statelog_frames.append(files_dict['USEPESTATELOG'])
            flightlog_frames.append(files_dict['USEPEFLIGHTLOG'])
            # Add lat, lon and altitude to conflog, loslog, cpalog
            cpalog_frames.append(addLatLonAlt(files_dict['USEPECPALOG'], files_dict['USEPESTATELOG'], 'cpa'))
            conflog_frames.append(addLatLonAlt(files_dict['USEPECONFLOG'], files_dict['USEPESTATELOG'], 'conf'))
            loslog_frames.append(addLatLonAlt(files_dict['USEPELOSLOG'], files_dict['USEPESTATELOG'], 'los'))

            statelog = pd.concat(statelog_frames)
            flightlog = pd.concat(flightlog_frames)
            cpalog = pd.concat(cpalog_frames)
            conflog = pd.concat(conflog_frames)
            loslog = pd.concat(loslog_frames)

    return conflog, cpalog, flightlog, loslog, statelog

def plotAltitudesFigure(alt_bins, ref_values, d2c2_values, percentual, y_label, title, use_case, fig_name):
    fig, ax = plt.subplots()
    
    a_heights, a_bins = np.histogram(ref_values, bins=alt_bins)
    b_heights, b_bins = np.histogram(d2c2_values, bins=alt_bins)
    
    width = (a_bins[1] - a_bins[0])/3
    
    if percentual:
        a_heights = 100*a_heights.astype(np.float32) / a_heights.sum()
        b_heights = 100*b_heights.astype(np.float32) / b_heights.sum()
    else:
        # Avegare considering the number of experiments
        a_heights = a_heights.astype(np.float32)/len(exp)
        b_heights = b_heights.astype(np.float32)/len(exp)
    
    ax.bar(a_bins[:-1]-width/2, a_heights, width=width, facecolor='cornflowerblue', label='Reference')
    ax.bar(b_bins[:-1]+width/2, b_heights, width=width, facecolor='seagreen', label='D2C2')
    #seaborn.despine(ax=ax, offset=10)
    ax.set_xlabel('Altitude [m]')
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()
    figure_path = Path(r"figures/exe{}/{}.png".format(use_case, fig_name))
    plt.savefig(figure_path)

def getPcolormeshData(row, alt_splits, lon_bins, lat_bins, ref_dataset, d2c2_dataset, scn):
    if scn == 'ref':
        dataset = ref_dataset
    else:
        dataset = d2c2_dataset
    
    if len(alt_splits) == 1:
        low_lim = 0
        high_lim = alt_splits[-1]
    else:
        if row == 0:
            low_lim = 0
            high_lim = alt_splits[row]
        else:
            low_lim = alt_splits[row-1]
            high_lim = alt_splits[row]
    
    x = dataset[(dataset[' alt'] > low_lim) & (dataset[' alt'] < high_lim) &
               (dataset[' lat'] > lat_min) & (dataset[' lat'] < lat_max) &
               (dataset[' lon'] > lon_min) & (dataset[' lon'] < lon_max)][' lon']
    y = dataset[(dataset[' alt'] > low_lim) & (dataset[' alt'] < high_lim) &
               (dataset[' lat'] > lat_min) & (dataset[' lat'] < lat_max) &
               (dataset[' lon'] > lon_min) & (dataset[' lon'] < lon_max)][' lat']
    
    H, xedges, yedges = np.histogram2d(x, y, bins=(lon_bins, lat_bins))

    H = H.T
    H /= len(exp)
    X, Y = np.meshgrid(xedges, yedges)
    
    
    
    return X, Y, H

def plotHeatMaps(lon_min, lon_max, lat_min, lat_max, lon_divisions, lat_divisions, alt_splits, 
                 ref_dataset, d2c2_dataset, 
                 figure_title, vmax,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name):
    
    '''
    This function plots the heat maps of different datasets
    
    Input:
        lon_min, lon_max, lat_min, lat_max
        lon_divisions, lat_divisions,
        alt_splits - list. If list contains only one value - assumed to be the upper limit of the complete airspace
                                            two or more values - considered to be the altitude splits
        ref_dataset, d2c2_dataset, 
        figure_title, vmax,
        figure_title_alinement, axis_labelpad,
        general_font, labels_font, titles_font,
        w_lon, w_lat
    Output:
        figure
    '''
    
    # Legend
    vmin = 0 
    vmax = vmax
    
    font = {'weight' : 'normal',
            'size'   : general_font}
    plt.rc('font', **font)

    lon_bins = [lon_min + x*(lon_max - lon_min )/lon_divisions for x in range(lon_divisions)] 
    lat_bins = [lat_min + x*(lat_max - lat_min)/lat_divisions for x in range(lat_divisions)]

    
    lon_size = (lon_max - lon_min)

    
    lat_size = (lat_max - lat_min) 

    n_rows = len(alt_splits)

    fig, axs = plt.subplots(n_rows,2,
                            figsize=(w_lon*lon_size+1,n_rows*w_lat*lat_size+1), 
                            gridspec_kw={'width_ratios': [4, 5]}, 
                            sharex = False, sharey = False)

    for row in range(n_rows):
        # Get legend value from d2c2 case
        X, Y, H = getPcolormeshData(row, alt_splits, lon_bins, lat_bins,ref_dataset, d2c2_dataset, 'd2c2')
        vmax = max(round(np.percentile(H, 96)), 1)
        
        X, Y, H = getPcolormeshData(row, alt_splits, lon_bins, lat_bins, ref_dataset, d2c2_dataset, 'ref')
        if n_rows == 1:
            im = axs[0].pcolormesh(X, Y, H, cmap=plt.cm.Reds, vmin=vmin, vmax=vmax)#vmax[row])
            axs[0].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=10)
            title = 'Reference - Complete airspace'
            axs[0].set_ylabel('lat [deg]',fontsize=labels_font, labelpad=axis_labelpad)
            axs[0].set_title(title,fontsize=titles_font,y=1.02)
            axs[0].set_aspect(1/np.cos(np.deg2rad(lat_min)))
        else:
            im = axs[row,0].pcolormesh(X, Y, H, cmap=plt.cm.Reds, vmin=vmin, vmax=vmax)#vmax[row])
            if row == 0:
                title = 'Reference - 0 to {} m'.format(alt_splits[row])
            elif row == n_rows-1:
                #axs[row,0].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=10)
                title = 'Reference - {} to {} m'.format(alt_splits[row-1], alt_splits[row])
            else:
                title = 'Reference - {} to {} m'.format(alt_splits[row-1], alt_splits[row])
            axs[row,0].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=3)
            axs[row,0].set_ylabel('lat [deg]',fontsize=labels_font, labelpad=axis_labelpad)
            axs[row,0].set_title(title,fontsize=titles_font,y=1.02)
            axs[row,0].set_aspect(1/np.cos(np.deg2rad(lat_min))) # to adjust latitude/longitude axis to their real size in meters
        #fig.colorbar(im, ax = axs[row,0])
        #print(H)

        X, Y, H = getPcolormeshData(row, alt_splits, lon_bins, lat_bins,ref_dataset, d2c2_dataset, 'd2c2')
        if n_rows == 1:
            im = axs[1].pcolormesh(X, Y, H, cmap=plt.cm.Reds, vmin=vmin, vmax=vmax)#vmax[row])
            axs[1].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=10)
            title = 'D2C2 - Complete airspace'
            #axs[1].set_ylabel('lat [deg]',fontsize=labels_font, labelpad=axis_labelpad)
            axs[1].set_title(title,fontsize=titles_font,y=1.02)
            axs[1].set_aspect(1/np.cos(np.deg2rad(lat_min)))
            fig.colorbar(im, ax = axs[1])
        else:
            im = axs[row,1].pcolormesh(X, Y, H, cmap=plt.cm.Reds, vmin=vmin, vmax=vmax)#vmax[row])
            if row == 0:
                title = 'D2C2 - 0 to {} m'.format(alt_splits[row])
            elif row == n_rows-1:
                #axs[row,1].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=10)
                title = 'D2C2 - {} to {} m'.format(alt_splits[row-1], alt_splits[row])
            else:
                title = 'D2C2 - {} to {} m'.format(alt_splits[row-1], alt_splits[row])
            axs[row,1].set_xlabel('lon [deg]' ,fontsize=labels_font, labelpad=3)
            axs[row,1].set_ylabel('lat [deg]',fontsize=labels_font, labelpad=axis_labelpad)
            axs[row,1].set_title(title,fontsize=titles_font,y=1.02)
            axs[row,1].set_aspect(1/np.cos(np.deg2rad(lat_min))) # to adjust latitude/longitude axis to their real size in meters
            fig.colorbar(im, ax = axs[row,1])
    #fig.suptitle(figure_title, fontsize=titles_font+2,y=figure_title_alinement)
    plt.subplots_adjust(left=0.1,
                        bottom=0.1,
                        right=0.9,
                        top=0.9,
                        wspace=0.2,
                        hspace=0.25)
    figure_path = Path(r"figures/exe{}/{}.png".format(use_case, fig_name))
    plt.savefig(figure_path)

## Batch analysis of the figures
The settings for the plots are below

In [ ]:
def analyseCase(density, folder_list, exercises, use_case):
    
    ##### Definded by user ######
    
    # 2D divisions for heat maps
    lon_divisions = 40
    lat_divisions = 40
    
    # Altitude splits heat maps
    alt_splits = [92, 225, 275]
    
    # Altitude aggregations histograms
    alt_bins = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275]
    
    # figures legend maximum
    #vmax_traf = [1000]*6
    vmax_traf = [1000, 1200, 500, 50, 50, 1500]
    vmax_conf = [25, 25, 15, 4, 4, 50]
    vmax_los = [20, 20, 10, 3, 3, 30]
    vmax_nmac = [10, 10, 5, 3, 3, 25]
    
    # Figures settings
    
    # Title and axis separation
    figure_title_alinement = 0.905
    axis_labelpad = 10
    
    # Fonts
    general_font = 14
    labels_font = 16
    titles_font = 18
    
    # Latitude and longitude sizes
    w_lon = 150    
    w_lat = 110
    
    
    (conflog_ref,
     cpalog_ref,
     flightlog_ref,
     loslog_ref,
     statelog_ref) = returnAggregatedDatasets(density, mode[0] ,folder_list, exercises)
    (conflog_sol,
     cpalog_sol,
     flightlog_sol,
     loslog_sol,
     statelog_sol) = returnAggregatedDatasets(density, mode[1] ,folder_list, exercises)
        
    #
    # Traffic distribution analysis
    #

    statelog_ref = statelog_ref[['SURVEILLANCE_' not in x for x in statelog_ref[' id']]]
    statelog_sol = statelog_sol[['SURVEILLANCE_' not in x for x in statelog_sol[' id']]]
    
    # # 2D
    ref_dataset = statelog_ref
    d2c2_dataset = statelog_sol
    figure_title = 'Traffic distribution'
    fig_name = 'traffic_distribution_{}_2d'.format(density)
    plotHeatMaps(lon_min, lon_max, lat_min, lat_max, 
                 lon_divisions, lat_divisions, 
                 alt_splits, 
                 ref_dataset, d2c2_dataset,
                 figure_title,
                 vmax_traf,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name)
    fig_name = 'traffic_distribution_{}_2d_complete'.format(density)
    plotHeatMaps(lon_min, lon_max, lat_min, lat_max, 
                 lon_divisions, lat_divisions, 
                 [275], 
                 ref_dataset, d2c2_dataset,
                 figure_title,
                 vmax_traf,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name)
    
    # # Altitudes
    ref_values = statelog_ref[' alt']
    d2c2_values = statelog_sol[' alt']
    y_label = 'Flight time [%]'
    title = 'Total time spent by drones at each altitude'
    fig_name = 'traffic_distribution_{}_alt'.format(density)
    plotAltitudesFigure(alt_bins, ref_values, d2c2_values, True, y_label, title, use_case, fig_name)
    
    #
    # Conflict distribution analysis
    #
    
    # # 2D
    ref_dataset = conflog_ref
    d2c2_dataset = conflog_sol
    figure_title = 'Conflict distribution'
    fig_name = 'conflict_distribution_{}_2d'.format(density)
    plotHeatMaps(lon_min, lon_max, lat_min, lat_max, 
                 lon_divisions, lat_divisions, 
                 [275], 
                 ref_dataset, d2c2_dataset,
                 figure_title,
                 vmax_conf,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name)
    
    # # Altitudes
    ref_values = conflog_ref[' alt']
    d2c2_values = conflog_sol[' alt']
    y_label = 'Number of drone pairs'
    title = 'Number of conflicts at each altitude '
    fig_name = 'conflict_distribution_{}_alt'.format(density)
    plotAltitudesFigure(alt_bins, ref_values, d2c2_values, False, y_label, title, use_case, fig_name)
    
    #
    # LoS distribution analysis
    #
    
    # # 2D
    ref_dataset = loslog_ref
    d2c2_dataset = loslog_sol
    figure_title = 'Loss of Separation distribution'
    fig_name = 'los_distribution_{}_2d'.format(density)
    plotHeatMaps(lon_min, lon_max, lat_min, lat_max, 
                 lon_divisions, lat_divisions, 
                 [275], 
                 ref_dataset, d2c2_dataset,
                 figure_title,
                 vmax_los,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name)
    
    # # Altitudes
    ref_values = loslog_ref[' alt']
    d2c2_values = loslog_sol[' alt']
    y_label = 'Number of drone pairs'
    title = 'Number of LoS at each altitude '
    fig_name = 'los_distribution_{}_alt'.format(density)
    plotAltitudesFigure(alt_bins, ref_values, d2c2_values, False, y_label, title, use_case, fig_name)
    
    #
    # NMAC distribution analysis
    #
    
    # # 2D
    ref_dataset = cpalog_ref
    d2c2_dataset = cpalog_sol
    figure_title = 'NMAC distribution'
    fig_name = 'nmac_distribution_{}_2d'.format(density)
    plotHeatMaps(lon_min, lon_max, lat_min, lat_max, 
                 lon_divisions, lat_divisions, 
                 [275], 
                 ref_dataset, d2c2_dataset,
                 figure_title,
                 vmax_nmac,
                 figure_title_alinement, axis_labelpad,
                 general_font, labels_font, titles_font,
                 w_lon, w_lat,
                 use_case, fig_name)
    
    # # Altitudes
    ref_values = cpalog_ref[' alt']
    d2c2_values = cpalog_sol[' alt']
    y_label = 'Number of drone pairs'
    title = 'Number of NMACs at each altitude '
    fig_name = 'nmac_distribution_{}_alt'.format(density)
    plotAltitudesFigure(alt_bins, ref_values, d2c2_values, False, y_label, title, use_case, fig_name)

## Plot the density desired

## Low density

In [ ]:
density = 'low'
analyseCase(density, folder_list, exercises, use_case)

## Medium density

In [ ]:
density = 'medium'
analyseCase(density, folder_list, exercises, use_case)

## High density

In [ ]:
density = 'high'
analyseCase(density, folder_list, exercises, use_case)